# Automatic segmentation of precipitate statistics using U-Net architecture

This code utilizes a U-Net deep learning architecture to automatically extract precipitate statistics from transmission electron microscopy (TEM) images. 
The code is developed as part of a master's thesis in applied physics, the code segments precipitates within the images, enabling the automatic measurement of precipitate length and cross-sections. 
By automating this process, it significantly accelerates the analysis of precipitate distributions, aiding in materials research and development. The code has been adapted from the Master Thesis of **Espen J. Gregory**, and has been restructered and further developed.

## Author:

**Kristian B. Thevik** - Developed for Master thesis in Physics 2025

## Note:
- It is recommended to have a GPU and the CUDA-version of Pytorch installed (However it is not required).
- Data can be loaded in two ways, either by directly uploading the .DM3 file, or converting the .DM3 to an image (.jpeg/.png) and manually selecting the calibration unit *nm_per_px*.
- U-Net documentation: https://arxiv.org/abs/1505.04597


### Imports and PyTorch initializationk

In [1]:
%matplotlib qt5
%load_ext autoreload
%autoreload 2
%pip install pandas

import cv2
import time
import torch
import numpy as np
import pandas as pd
import tkinter as tk
#import _dm3_lib as dm
from pathlib import Path
from PIL import Image
from itertools import product
from tkinter import filedialog
#from u_net_pytorch import UNet
from skimage import measure, color, io
from skimage.segmentation import clear_border
from pathlib import Path

"""PyTorch Initialization"""
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark     = True
torch.manual_seed(0)
torch.cuda.manual_seed(0)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("Device type: %s"%(device))


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Device type: cpu


In [2]:
# Using DataSetEvaluator
from testMaster.DatasetEvaluator import UNETEvaluator

# If files in the dataset are .dm3 files, nm_per_px will be extracted automatically and replace the 
# existing nm_per_px value in the list. It works to have some files as dm3 and some as .jpg/.png.
# Always give nm_per_px as a list with lenght equal to number of files to be evaluated. First value corresponds
# to nm_per_px value for the first image and so on. For dm3 files an arbitrary value can be given, it will be replaced
# either way.


nm_per_px = [0.069661] * 8 #Cross
#nm_per_px = [0.16685] * 8 #Length

# Example usage:
this_dir = Path.cwd()
dataset_path = this_dir.parent / "data" / "var3_NA_5h185C_length"
model_path = this_dir.parent / "data" / "models" / "length_unet.pth"


unet_evaluator = UNETEvaluator(
    dataset_dir = dataset_path,
    model = model_path,
    nm_per_px = nm_per_px,
    type = 'length',
    device = 'cpu'
)

unet_evaluator.statistics()



C:\Users\krist\Documents\masterRepo\src\testMaster\DatasetEvaluator.py:312: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.checkpoint = torch.load(model, map_location=to

 Unet Model Loaded


c:\Users\krist\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\krist\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Average: 26.56 nm, STDev: 16.28 nm, Number counted: 963, STDev of mean: 0.52 nm, Number density: 0.0009617nm^-2
664 [17.0089999962, 12.0, 13.005, 9.00100000381, 5.00900000095, 14.0089999962, 6.0099999905, 11.0089999962, 10.0060000038, 7.00300000191, 10.0089999962, 15.0030000019, 10.0010000038, 10.0069999981, 9.005, 13.0019999981, 13.0039999962, 11.0039999962, 13.0, 8.00715000153, 5.00699999809, 6.00400000095, 7.0, 10.0010000038, 11.0069999981, 8.00199999809, 8.00800000191, 8.00100000381, 7.00300000191, 8.0, 23.0070000076, 11.0010000038, 15.0, 16.005, 12.0060000038, 12.0080000019, 9.00300000191, 8.0, 12.0030000019, 11.0019999981, 14.0, 8.005, 9.0, 12.005, 12.0019999981, 16.0020000076, 12.0030000019, 11.005, 7.00400000095, 21.005, 15.005, 13.0, 7.00699999809, 9.00399999619, 8.00100000381, 8.00600000381, 12.0039999962, 7.00199999809, 9.005, 12.0, 11.0080000019, 13.0060000038, 12.0039999962, 20.0020000076, 11.0080000019, 21.0, 9.00800000191, 11.0, 11.0060000038, 6.00800000191, 17.002000007